# API de OpenAI

In [1]:
import os
from openai import OpenAI
import requests
import json

os.environ["OPENAI_API_KEY"] = "ACA VA EL API KEY"
client = OpenAI()

### Ejemplo de uso de la API de OpenAI


In [ ]:

prompt = '''Generar un poema breve sobre "cuenta dni", la billetera virtual de banco provincia'''

## la principal forma de interactuar con la API de OpenAI es utilizando la api de Responses.
## esta api nos permite generar texto a partir de un prompt.

response = client.responses.create(
    model="o4-mini",
    input=prompt
)

print(response.output_text)

Cuenta DNI, llave de conexión,  
pulso digital que agiliza mi acción.  
En cada QR brota la libertad,  
sin efectivo ni pausa, pura agilidad.  
Banco Provincia, aliado fiel,  
en tu billetera mi ahorro a un nuevo nivel.


Vamos a ver que tipo de cosas podemos hacer con la API de OpenAI que tengan que ver especificamente con datos. 

### Clasificación zero-shot de texto

In [18]:
# Ejemplo: Clasificación de sentimiento en reviews
## podemos clasificar el sentimiento de un texto, es decir, si es positivo, neutral o negativo.
## antes era necesario tener un dataset de ejemplos para poder entrenar un clasificador. 
## Ahora podemos hacerlo de manera directa con un LLM, que funciona bastante bien.

reviews = [
    "El producto es excelente, superó mis expectativas.",
    "No estoy satisfecho con la compra, llegó dañado.",
    "Funciona bien, pero la entrega tardó mucho.",
    "¡Increíble! Lo recomendaré a todos mis amigos.",
    "Pésima atención al cliente, no resolverían mi problema.",
]

instruccion = "Clasifica el siguiente texto como 'positivo', 'neutral' o 'negativo'. Responde solo con una de estas tres palabras.\n\n"

for review in reviews:
    prompt = instruccion + f"Texto: {review}"
    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
    )
    sentimiento = response.output_text
    print(f"Review: {review}")
    print(f"Sentimiento: {sentimiento}")
    print("-" * 50)


Review: El producto es excelente, superó mis expectativas.
Sentimiento: Positivo
--------------------------------------------------
Review: No estoy satisfecho con la compra, llegó dañado.
Sentimiento: Negativo
--------------------------------------------------
Review: Funciona bien, pero la entrega tardó mucho.
Sentimiento: Neutral
--------------------------------------------------
Review: ¡Increíble! Lo recomendaré a todos mis amigos.
Sentimiento: Positivo
--------------------------------------------------
Review: Pésima atención al cliente, no resolverían mi problema.
Sentimiento: Negativo
--------------------------------------------------


### Estructurar dato no estructurado

In [19]:
## vamos a tomar un texto de una noticia y vamos a extraer la información de la noticia.
## y lo vamos a devolver como un json. De esta forma podemos convertir un dato no estructurado en un dato estructurado.

import openai
import requests
import json

client = openai.OpenAI()

# Obtener texto de una página web (ejemplo: noticia de Argentina)
url = "https://www.infobae.com/economia/2025/04/25/el-gobierno-retoma-los-pagos-al-fmi-y-debera-devolver-por-intereses-usd-2000-millones-este-ano/"
response = requests.get(url)
texto_noticia = response.text[:2000]  # Tomamos solo los primeros 2000 caracteres


In [32]:

# Prompt unificado para el modelo 'responses'
prompt = f"""
Eres un asistente que extrae información estructurada de texto no estructurado y la devuelve como JSON.

Extrae la siguiente información del texto como un objeto JSON:
- titulo (string)
- autor (string)
- fecha (string en formato YYYY-MM-DD)
- entidades_principales (array de strings)
- resumen (string de 2 oraciones)
- categoria (string)

Texto:
{texto_noticia}
"""

# Llamada a la nueva API
response = client.responses.create(
    model="gpt-4o-mini",
    input=prompt
)



In [36]:
salida = response.output_text
salida = salida.replace("```json", "").replace("```", "")


In [38]:
try:
    datos_estructurados = json.loads(salida)
    print(json.dumps(datos_estructurados, indent=2, ensure_ascii=False))
except json.JSONDecodeError:
    print("⚠️ No se pudo parsear la respuesta como JSON:")
    print(salida)


{
  "titulo": "El Gobierno retoma los pagos al FMI y deberá devolver por intereses USD 2.000 millones este año - Infobae",
  "autor": "Infobae",
  "fecha": "2023-10-01",
  "entidades_principales": [
    "Gobierno",
    "FMI",
    "BCRA"
  ],
  "resumen": "La semana próxima vencen USD 630 millones y el calendario incluye nuevas obligaciones en agosto y noviembre. Economía recibió $11,7 billones desde el BCRA para afrontar deuda en pesos y dólares.",
  "categoria": "Economía"
}
